## This notebook shows how to run a whole lot of propagations

In [2]:
from adam import Batch
from adam import PropagationParams
from adam import OpmParams
from adam import BatchRunManager
from adam import Service

import datetime
import os

This sets up a Service which uses the given token and URL to provide authorized access through the server through several wrapped modules. It also creates a project for you to work in that will be used for the rest of the notebook. Be sure to run service.teardown() when finished. 

If you don't have a token, see auth_demo to get one.

Normally instead of 'ffffffff-ffff-ffff-ffff-ffffffffffff' you would use your team workspace project that was made for you by the ADAM team.

In [3]:
url = "https://pro-equinox-162418.appspot.com/_ah/api/adam/v1"
 # Reads your token from a file in current directory. For instructions on getting a token, see auth_demo notebook.
token = open(os.getcwd() + '/token.txt').read()
service = Service()
service.setup(url, token, 'ffffffff-ffff-ffff-ffff-ffffffffffff')
working_project = service.get_working_project()

ConnectionError: HTTPSConnectionPool(host='pro-equinox-162418.appspot.com', port=443): Max retries exceeded with url: /_ah/api/adam/v1/me?token=ik6O46g4ZDThQjEKrvp5ZQo36ai2 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x10596f7f0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))

This function creates a propagation object. For now, it just creates a dummy object with slight variations to avoid creating duplicate objects. For detailed information on Batch creation, see test_single_batch_run notebook.

In [6]:
def get_batch(i):
    state_vec = [130347560.13690618,
                 -74407287.6018632,
                 -35247598.541470632,
                 23.935241263310683,
                 27.146279819258538,
                 10.346605942591514]
    
    start_time = '2018-02-04T00:00:00Z'
    start_time = datetime.datetime(2018, 2, 21, 0, 0, 0, 123456)
    end_time = datetime.datetime(2028, 2, 21, 0, 0, 0, 123456)  # Ten years later.
    
    return Batch(PropagationParams({
        'start_time': start_time.isoformat() + 'Z',
        'end_time': end_time.isoformat() + 'Z',
        'project_uuid': working_project.get_uuid(),
    }), OpmParams(dict(asteroid_params, **{
        'epoch': start_time.isoformat() + 'Z',
        'state_vector': state_vec,
        'mass': 500 + i,  # Create a small variation between batches.
    })))

This creates a large number of batch runs and propagates them.

In [7]:
num_batches = 1000
batches = [get_batch(i) for i in range(num_batches)]

batch_run_manager = BatchRunManager(service.get_batches_module(), batches)
batch_run_manager.run()

NameError: name 'working_project' is not defined

To view the status of your runs while they're propagating, run this.

In [ ]:
status = batch_run_manager.get_latest_statuses()
status_counts = {s: len(status[s]) for s in status}
print(tabulate.tabulate(status_counts, headers="keys", tablefmt="fancy_grid"))

Once the batches are all done propagating, you can view their results.

In [ ]:
for batch in batches:
    if batch.get_calc_state() == 'FAILED':
        error = 'unknown error'
        if batch.get_results() is not None and batch.get_results().get_parts()[0] is not None:
            error = batch.get_results().get_parts()[0].get_error()
        print('Batch %s failed: %s' % (batch.get_uuid(), error))
    else:
        final_state_vector = batch.get_results().get_final_state_vector()
        print('Batch %s ended at %s' % (batch.get_uuid(), final_state_vector))

That's it! Please clean up by running the following:

In [1]:
service.teardown()

NameError: name 'service' is not defined